In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train
from itertools import chain

%aimport QANet_wo_charemb_glove_unk

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
%%time
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/'

data = np.load(squad_path + 'train_with_glove_vocab.pkl')
glove_embs = np.load(glove_path + 'glove.6B.300d.npy')
voc_size = glove_embs.shape[0]

# Some answers get broken in the process of tokenization, because some answer words are not properly split.
def filter_broken_answers(data):
    return [d for d in data if d[0]]

print 'Examples total:', len(data)

data = filter_broken_answers(data)

print 'Working examples:', len(data)

Examples total: 87599
Working examples: 86474
CPU times: user 9 s, sys: 190 ms, total: 9.19 s
Wall time: 9.2 s


In [3]:
trim = 300
data = [d[:2] + [d[2][:trim]] for d in data if max(d[0][0]) < trim]
print 'Trimmed examples:', len(data)

Trimmed examples: 86355


In [4]:
data_dev = np.load(squad_path + 'dev_with_glove_vocab.pkl')

## QANet

In [8]:
%%time

update_fn = lambda l, p: L.updates.adam(l, p)

qa_net = QANet_wo_charemb_glove_unk.QANet(voc_size=voc_size,
                                          emb_size=300,
                                          rec_size=300,
                                          emb_init=glove_embs,
                                          train_inds=[],
                                          update_fn=update_fn)

Building the model...
Using custom update_fn.
Compiling theano functions...
Done
CPU times: user 1min 17s, sys: 2.1 s, total: 1min 19s
Wall time: 1min 22s


In [14]:
qa_net.load_params('trained_models/glove_vocab/simplified_all_fixed3_1ep.npz', glove_embs)

In [6]:
# 1ep train set na słowniku z glove

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# zanurzenia z glove, unk to średnie słowo, nie trenujemy zanurzeń

qa_net.train_one_epoch(data, 20, log_interval=5)

Done 5 batches in 4.76s	training loss:	9.284095
Done 10 batches in 9.43s	training loss:	8.997582
Done 15 batches in 13.89s	training loss:	8.464465
Done 20 batches in 18.78s	training loss:	8.095969
Done 25 batches in 23.56s	training loss:	7.775052
Done 30 batches in 28.26s	training loss:	7.550472
Done 35 batches in 32.42s	training loss:	7.391315
Done 40 batches in 37.18s	training loss:	7.235471
Done 45 batches in 41.60s	training loss:	7.132887
Done 50 batches in 46.32s	training loss:	7.046891
Done 55 batches in 50.74s	training loss:	6.958380
Done 60 batches in 55.21s	training loss:	6.879590
Done 65 batches in 60.05s	training loss:	6.823590
Done 70 batches in 65.01s	training loss:	6.789325
Done 75 batches in 68.90s	training loss:	6.755375
Done 80 batches in 73.55s	training loss:	6.680267
Done 85 batches in 78.50s	training loss:	6.642193
Done 90 batches in 83.20s	training loss:	6.574215
Done 95 batches in 88.16s	training loss:	6.534975
Done 100 batches in 92.41s	training loss:	6.512547
Do

3.4495795286423867

In [7]:
qa_net.save_params('trained_models/glove_unks/simplified_glove_unks_all_fixed2_1ep')

In [7]:
# 1ep train set na słowniku z glove

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# zanurzenia z glove, unk to średnie słowo, nie trenujemy zanurzeń

qa_net.train_one_epoch(data, 20, log_interval=5)

Done 5 batches in 5.10s	training loss:	8.929138
Done 10 batches in 9.82s	training loss:	8.647834
Done 15 batches in 14.73s	training loss:	8.301548
Done 20 batches in 19.32s	training loss:	8.065414
Done 25 batches in 23.68s	training loss:	7.757741
Done 30 batches in 28.15s	training loss:	7.532715
Done 35 batches in 32.81s	training loss:	7.467324
Done 40 batches in 37.12s	training loss:	7.296044
Done 45 batches in 42.06s	training loss:	7.128645
Done 50 batches in 46.62s	training loss:	7.039674
Done 55 batches in 50.88s	training loss:	6.948180
Done 60 batches in 55.94s	training loss:	6.881593
Done 65 batches in 60.44s	training loss:	6.819214
Done 70 batches in 65.20s	training loss:	6.788998
Done 75 batches in 69.92s	training loss:	6.759360
Done 80 batches in 74.74s	training loss:	6.708144
Done 85 batches in 79.59s	training loss:	6.646897
Done 90 batches in 84.35s	training loss:	6.621784
Done 95 batches in 88.97s	training loss:	6.583456
Done 100 batches in 93.74s	training loss:	6.558462
Do

3.3460889364562991

In [8]:
qa_net.save_params('trained_models/glove_unks/simplified_glove_unks_all_fixed_1ep')

### QANet tests

In [ ]:
'''
1ep all fixed : {"f1": 63.93851222573222, "exact_match": 53.33964049195837}
1ep train unk : {"f1": 59.56790117987117, "exact_match": 49.10122989593188}
'''

In [15]:
def predict_spans(data, beam=10, batch_size=10):
    num_examples = len(data)
    
    start_probs = qa_net.get_start_probs(data, batch_size)
    best_starts = start_probs.argpartition(-beam, axis=1)[:, -beam:].astype(np.int32)
    
    scores = start_probs[np.arange(num_examples)[:, np.newaxis], best_starts]
    scores = np.tile(scores[:, np.newaxis], (beam, 1)).transpose(0, 2, 1)
    
    best_ends_all = []
    for i in xrange(beam):
        end_probs = qa_net.get_end_probs(data, best_starts[:, i], batch_size)
        best_ends = end_probs.argpartition(-beam, axis=1)[:, -beam:]
        scores[:, i, :] *= end_probs[np.arange(num_examples)[:, np.newaxis], best_ends]
        best_ends_all.append(best_ends)
        
    best_ends_all = np.hstack(best_ends_all)
        
    scores = scores.reshape(num_examples, beam**2)
    best_spans = scores.argmax(axis=1)
    starts = [i / beam for i in best_spans]
    
    starts = best_starts[np.arange(num_examples), starts]
    ends = best_ends_all[np.arange(num_examples), best_spans]
    
    return starts, ends

In [16]:
%%time
predicted_spans = []
batch_size = 10

idx = 0
while idx < len(data_dev):
    spans = predict_spans(data_dev[idx:idx + batch_size], beam=1)
    predicted_spans.append(np.vstack(spans))
    idx += batch_size
    if not idx % 1000:
        print idx, 'examples'
        
print 'Predictions done'
    
predicted_spans = np.hstack(predicted_spans).T

1000 examples
2000 examples
3000 examples
4000 examples
5000 examples
6000 examples
7000 examples
8000 examples
9000 examples
10000 examples
Predictions done
CPU times: user 13min 27s, sys: 28min 45s, total: 42min 12s
Wall time: 6min 51s


In [17]:
np.savez('evaluate/glove_vocab/dev_with_glove_vocab_predictions_simplified_all_fixed3_1ep', predicted_spans)